In [103]:
import json
import numpy as np
import os
import pandas as pd

In [104]:
behavior_names = ['XY_05232017', 'NG_05242017', 'NK_05242017', 'DN_05302017', 'MS_05312017','ST_05312017',
                  'CU_06012017', 'AM_06012017', 'EA_06062017', 'CM_06062017', 'LC_06072017', 'TP_06082017',
                  'TC_10112017', 'CF_10122017', 'VS_10172017', 'NP_10202017', 'MH_11272017', 'DB_11282017',
                  'RR_11282017', 'EL_11292017', 'VD_11292017', 'JS_11302017', 'HR_12012017', 'MA_12012017',
                  'FD_12012017','CS_12042017','KD_01092018','IR_01092018','AA_01112018','AM2_01112018',
                  'HZ_01242018','DA_02072018','CV_02092018','MS2_02192018','EC_02212018','TB_02212018',
                  'JW_02232018','LL_02232018','KL_02232018']

behavior_names_short = [name.split('_')[0] for name in behavior_names]

N = len(behavior_names)

direc = 'Y:/bas/data/behavior/'

In [105]:
data_output = [json.load(open(direc + 'output/FreechoiceData' + name + '.txt')) for name in behavior_names]

In [106]:
data_db = [json.load(open(direc + 'database/Database' + name + '.json')) for name in behavior_names_short]

In [107]:
fnames = [direc + "console/console" + name + ".log" for name in behavior_names]
data_console = [open(name,"r").readlines() if os.path.isfile(name) else [] for name in fnames]

In [108]:
X = np.array([[[int(e['response']),int(e['trial_id'])] for e in d if 'response' in e][-216:] for d in data_db[3:]])

In [109]:
choices = np.empty([36,216],dtype=int)
for i in range(36):
    choices[i,X[i,:,1]]=X[i,:,0]

In [110]:
np.max(X[:,:,1])

215

In [111]:
np.mean(np.array([d['choices'] for d in data_output[3:]])==choices)

1.0

In [112]:
#helper functions that encode boards as and moves as integers for the model fitting pipeline
def encode_board(pieces):
    return np.sum([2**i for i,p in enumerate(pieces) if p=='1']).astype(np.int64)

def encode_move(m):
    return 2**int(m)



In [169]:
direc = 'C:/Users/svo/Documents/FourinarowData/Data/freechoice/'
if not os.path.exists(direc + 'splits/'):
    os.mkdir(direc + 'splits/')
for i,d in enumerate(data_db[3:]):
    if not os.path.exists(direc + 'splits/' + str(i+1)):
        os.mkdir(direc + 'splits/' + str(i+1))
    d_valid = [e for e in d if 'response' in e][-216:]
    d_valid = [e for e in d_valid if e['response']!='99']
    df = pd.DataFrame([(encode_board(e['bp']),
                             encode_board(e['wp']),
                             'Black' if e['color']=='0' else 'White',
                             encode_move(e['response']),
                             e['rt'],
                             e['user_name'])
                            for e in d_valid],
                 columns=['bp','wp','color','m','rt','user_name'])
    df.loc[df['color']=='Black','bp']-=df[df['color']=='Black']['m']
    df.loc[df['color']=='White','wp']-=df[df['color']=='White']['m']
    df['group'] = (5*(np.random.permutation(len(df))/len(df))).astype(int)+1
    df = df[['bp','wp','color','m','rt','group','user_name']]
    with open(direc + 'splits/' + str(i+1) + '/data.csv','w') as f:
        f.write(df.to_csv(None, index = False, header=False,sep='\t',line_terminator ='\n')[:-1])
    for g in range(1,6):
        with open(direc + 'splits/' + str(i+1) + '/' + str(g) + '.csv','w') as f:
            f.write(df[df['group']==g].to_csv(None, index = False, header=False,sep='\t',line_terminator ='\n')[:-1])
    

In [170]:
(df['bp']%(2*df['m']))>=df['m']

0      False
1      False
2      False
3      False
4      False
       ...  
209    False
210    False
211    False
212    False
213    False
Length: 214, dtype: bool